# Read and plot Rank statistics
- Run all cells to generate plots for the randomised simulations.
- Generated images will be saved in `Plots`

In [ ]:
import pandas as pd
import PlotGWorld
import os
import pprint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# from matplotlib import font_manager
# # Rebuild font cache
# font_manager._load_fontmanager(try_read_cache=False)

# # Configure matplotlib to use LM Roman for ALL text
# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': ['LMRoman10'],
#     'font.sans-serif': ['LMRoman10'],  # Force serif even for sans-serif
#     'font.monospace': ['LMMono10'],
#     'mathtext.fontset': 'cm',
#     'axes.unicode_minus': False,
# })

sns.set_theme(
    style="ticks",
    rc={
        'font.family': 'serif',
        # 'font.serif': ['LMRoman10'],
        # 'font.sans-serif': ['LMRoman10'],
    }
)

fig_folder = 'Plots'
results_folder='gFeAR_Results'

In [ ]:


files = [f for f in os.listdir(results_folder) if os.path.isfile(os.path.join(results_folder, f))]

taus_df = None
ns_assertive_df = None

intersection_files = [file for file in files if 'Intersection12by12' in file]


for f,file in enumerate(intersection_files):

    print(f'File number {f} : {file}')
    
    ranks_df = pd.read_pickle(os.path.join(results_folder,file))

    # Get the number of agents and iterations
    n_agents = len(ranks_df['Action4Agents'][0])
    n_iterations = ranks_df.shape[0]//n_agents
    # Get short sceanrio_name
    scenario_name = ranks_df['scenario_name'][0].split('-')[0]

    # Finding affected agent per row
    ranks_df['affected'] = ranks_df['tier_ranks'].apply(lambda x: np.where(x == 0)[0][0])

    # Calculating the median and mean distance of other agents from the affected agent.
    ranks_df['median_manhattan_dist'] = ranks_df.apply(
    lambda row: np.median(
        (dists := np.abs(row['AgentLocations'] - row['AgentLocations'][row['affected']]).sum(axis=1))[dists > 0]
    ),
    axis=1)
    ranks_df['mean_manhattan_dist'] = ranks_df.apply(
    lambda row: np.mean(
        (dists := np.abs(row['AgentLocations'] - row['AgentLocations'][row['affected']]).sum(axis=1))[dists > 0]
    ),
    axis=1)

    # Count the number of assertive agents identified
    ranks_df['n_assertive_tier'] = ranks_df['tier_ranks'].apply(lambda x: np.sum((x > 0) & (x < n_agents+1)))
    ranks_df['n_assertive_fear'] = ranks_df['fear_ranks'].apply(lambda x: np.sum((x > 0) & (x < n_agents+1)))
    ranks_df['n_assertive_shap'] = ranks_df['shap_ranks'].apply(lambda x: np.sum((x > 0) & (x < n_agents+1)))

    ranks_df['n_assertive_tier-fear'] = ranks_df['n_assertive_tier'] - ranks_df['n_assertive_fear']
    ranks_df['n_assertive_shap-fear'] = ranks_df['n_assertive_shap'] - ranks_df['n_assertive_fear']
    ranks_df['n_assertive_shap-tier'] = ranks_df['n_assertive_shap'] - ranks_df['n_assertive_tier']

    # Get the diffs in n_assertive and get melted df
    n_assertive_df_ = ranks_df[['n_assertive_tier-fear',
                                'n_assertive_shap-fear',
                                'n_assertive_shap-tier',
                                'n_assertive_tier',
                                'median_manhattan_dist',
                                'mean_manhattan_dist']]
    n_assertive_df = pd.melt(n_assertive_df_,
                             id_vars=['median_manhattan_dist','mean_manhattan_dist','n_assertive_tier'],
                             value_vars=['n_assertive_tier-fear','n_assertive_shap-fear','n_assertive_shap-tier'],
                             var_name = 'comparison',
                             value_name='diff_n_assertive'
                             )
    # Clean up the letter column to remove 'n_assertive_' prefix
    n_assertive_df['comparison'] = n_assertive_df['comparison'].str.replace('n_assertive_', '')
    n_assertive_df['comparison'] = n_assertive_df['comparison'].str.replace('fear','iFeAR')
    n_assertive_df['comparison'] = n_assertive_df['comparison'].str.replace('tier','gFeAR(Tier)')
    n_assertive_df['comparison'] = n_assertive_df['comparison'].str.replace('shap','gFeAR(Shapley)') 
    n_assertive_df['comparison'] = n_assertive_df['comparison'].str.replace('_',' - ')

    # Get the taus and get melted df
    # ---------------------------------------------------------------------------------------------------------------------- #
    # tau_df_ = ranks_df[['tau_fear_tier','tau_fear_shap','tau_tier_shap','median_manhattan_dist','mean_manhattan_dist']].dropna()
    tau_df_ = ranks_df[['tau_fear_tier','tau_fear_shap','tau_tier_shap','median_manhattan_dist','mean_manhattan_dist']]
    tau_df = pd.melt(tau_df_,
                     id_vars=['median_manhattan_dist','mean_manhattan_dist'],
                     value_vars=['tau_fear_tier','tau_fear_shap','tau_tier_shap'],
                     var_name = 'comparison',
                     value_name='tau'
                    )
    # Clean up the letter column to remove 'tau_' prefix
    tau_df['comparison'] = tau_df['comparison'].str.replace('tau_', '')
    # tau_df['comparison'] = tau_df['comparison'].str.replace('fear_shap', 'shap_fear')
    tau_df['comparison'] = tau_df['comparison'].str.replace('fear','iFeAR')
    tau_df['comparison'] = tau_df['comparison'].str.replace('tier','gFeAR(Tier)')
    tau_df['comparison'] = tau_df['comparison'].str.replace('shap','gFeAR(Shapley)') 
    tau_df['comparison'] = tau_df['comparison'].str.replace('_',' - ')
    
    # get specifier from filename
    specifier = file.split('-')[-5].split('_')[0].capitalize()

    # Add file identification to melted df   
    tau_df['file'] = specifier
    n_assertive_df['file'] = specifier

    # Exclude NaN values
    tau_df = tau_df.dropna()

    # Concatinate dfs
    if ns_assertive_df is None:
        ns_assertive_df = n_assertive_df
    else:
        ns_assertive_df = pd.concat([ns_assertive_df, n_assertive_df], ignore_index=True)
    
    # Concatinate dfs
    if taus_df is None:
        taus_df = tau_df
    else:
        taus_df = pd.concat([taus_df, tau_df], ignore_index=True)

# Make boxplots
ax= sns.boxplot(taus_df, y='tau', x='file', hue='comparison', gap=0.1, palette="vlag" )
ax.tick_params(axis='x', labelrotation=45)
PlotGWorld.make_axes_gray(ax)

_,_ = plt.subplots()
ax= sns.boxplot(ns_assertive_df, y='diff_n_assertive', x='file', hue='comparison', gap=0.1, palette="vlag" )
ax.tick_params(axis='x', labelrotation=45)
PlotGWorld.make_axes_gray(ax)

In [ ]:
g = sns.FacetGrid(ns_assertive_df[ns_assertive_df['comparison']=='gFeAR(Tier)-iFeAR'], col='comparison', aspect=1.5, height=5)

# Function to plot sized scatter points with color by file
def plot_sized_scatter(data, **kwargs):
    # Get counts for this facet, grouped by file
    facet_counts = data.groupby(['median_manhattan_dist', 'diff_n_assertive', 'file', 'comparison']).size().reset_index(name='count')

    color_map = {
        'Aggressive':'deeppink',
        'Nuanced': 'deepskyblue',
        'Random': 'gold'}
    
    # Plot each file with a different color
    for file_name in ['Aggressive', 'Nuanced', 'Random']:
        file_data = facet_counts[facet_counts['file'] == file_name]
        plt.scatter(file_data['median_manhattan_dist'], 
                    file_data['diff_n_assertive'],
                    color=color_map[file_name],
                    s=1+file_data['count']*5,  # Scale factor - adjust as needed,
                    alpha=0.7, marker='o', label=file_name.capitalize())

    # # Outlines on top
    # for file_name in ['Random', 'Nuanced', 'Aggressive']:
    #     file_data = facet_counts[facet_counts['file'] == file_name]
    #     plt.scatter(file_data['median_manhattan_dist'], 
    #                 file_data['diff_n_assertive'],
    #                 edgecolor=color_map[file_name],
    #                 facecolor='none',
    #                 s=1+file_data['count']*5,  # Scale factor - adjust as needed,
    #                 alpha=0.5, marker='o', label='_nolegend_')

g.map_dataframe(plot_sized_scatter)


# Add the max line
def plot_max_line(data, **kwargs):
    max_data = data.groupby('median_manhattan_dist')['n_assertive_tier'].max().reset_index()
    max_data = max_data.sort_values('median_manhattan_dist')
    plt.plot(max_data['median_manhattan_dist'], max_data['n_assertive_tier'], label='Maximum\nnumber of\nassertive agents',
             color='lightgray', linewidth=4, alpha=0.7, zorder=10)
g.map_dataframe(plot_max_line)


# Plot deduplicated points with '+' markers
def plot_unique_plus_markers(data, **kwargs):
    unique_data = data[['median_manhattan_dist', 'n_assertive_tier']].drop_duplicates()
    plt.scatter(unique_data['median_manhattan_dist'], 
                unique_data['n_assertive_tier'],
                marker='+', 
                linewidth=1,
                s=50, 
                color='gray', 
                alpha=0.9, 
                zorder=0,
                label='Number of \nassertive agents\nper instance')

g.map_dataframe(plot_unique_plus_markers)                       

# Add legend
g.add_legend(loc='upper right', labelspacing = 1)


plt.tight_layout()
PlotGWorld.make_axes_gray(g.axes[0,0])

ax = g.axes[0,0]
ax.set_title('Comparing assertive agents identified by \n gFeAR(Tier) and iFeAR')
ax.set_ylabel(r'$\Delta_j^{\mathrm{Assertive}}$')
ax.set_xlabel('Median Manhattan distance to affected agent')

fig_name = 'Comparing-n-assertive-tier-fear.pdf'
fig_path = os.path.join(fig_folder,fig_name)
plt.savefig(fig_path)

fig_name = 'Comparing-n-assertive-tier-fear.png'
fig_path = os.path.join(fig_folder,fig_name)
plt.savefig(fig_path)

In [ ]:
sns.set_context("paper", font_scale=2)

file_palette = {
        'Aggressive':'deeppink',
        'Nuanced': 'deepskyblue',
        'Random': 'gold'}

count_marker_scale=4
count_marker_min=5

# Create figure with 3 subplots in a row (or column)
fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True) 

# ===== PLOT 1: Scatter plot with sized points =====
ax1 = axes[0]
data1 = ns_assertive_df[ns_assertive_df['comparison']=='gFeAR(Tier)-iFeAR']
# Prepare data
facet_counts = data1.groupby(['median_manhattan_dist', 'diff_n_assertive', 'file', 'comparison']).size().reset_index(name='count')
# Get unique distance values and create a mapping to categorical positions
unique_dists = sorted(facet_counts['median_manhattan_dist'].unique())
dist_to_pos = {dist: i for i, dist in enumerate(unique_dists)}
color_map = {
    'Aggressive':'deeppink',
    'Nuanced': 'deepskyblue',
    'Random': 'gold'
}
# Add max line
max_data = data1.groupby('median_manhattan_dist')['n_assertive_tier'].max().reset_index()
max_data = max_data.sort_values('median_manhattan_dist')
x_positions = max_data['median_manhattan_dist'].map(dist_to_pos)
line, = ax1.plot(x_positions, max_data['n_assertive_tier'], 
         label=r'Max($n^\mathrm{Assertive}_{j,\,\mathrm{gFeAR}}$)',
         color='lightgray', linewidth=4, alpha=0.7, zorder=10)

# Plot scatter points using categorical positions
scatter_handles = []
for file_name in ['Aggressive', 'Nuanced', 'Random']:
    file_data = facet_counts[facet_counts['file'] == file_name]
    x_positions = file_data['median_manhattan_dist'].map(dist_to_pos)
    sc = ax1.scatter(x_positions, 
                file_data['diff_n_assertive'],
                color=color_map[file_name],
                s=count_marker_min+file_data['count']*count_marker_scale,
                alpha=0.7, marker='o', label=file_name.capitalize())
    scatter_handles.append(sc)

# Plot unique plus markers
unique_data = data1[['median_manhattan_dist', 'n_assertive_tier']].drop_duplicates()
x_positions = unique_data['median_manhattan_dist'].map(dist_to_pos)
ax1.scatter(x_positions, 
            unique_data['n_assertive_tier'],
            marker='+', 
            linewidth=1,
            s=50, 
            color='gray', 
            alpha=0.9, 
            zorder=0,
            label='_nolegend_')

ax1.set_title('')
ax1.set_ylabel(r'$n^\mathrm{Assertive}_{j,\,\mathrm{gFeAR}} - n^\mathrm{Assertive}_{j,\,\mathrm{iFeAR}}$'+'\n\n'+
               r'= $\Delta_j^{\mathrm{Assertive}}$', fontsize=28)

# Get min and max counts for size legend
# max_count = facet_counts['count'].max()
# count_range = [1] + list(range(200, max_count, 300))
count_range = [1, 200, 400]

# Create dummy scatter plots for size legend
size_handles = []
size_labels = []
for count_val in count_range:
    dummy = ax1.scatter([], [], s=count_marker_min+count_val*count_marker_scale, color='gray', alpha=0.7, marker='o')
    size_handles.append(dummy)
    size_labels.append(f'{count_val}')

# Create first legend for scatter points with title
leg1 = ax1.legend(handles=scatter_handles + size_handles, 
                  labels=['Aggressive', 'Nuanced', 'Random'] + size_labels,
                  title=r'$\Delta_j^{\mathrm{Assertive}}$ counts',
                  loc='upper right',
                  ncols=2,
                  labelspacing=0.4, 
                  frameon=False)

leg1.get_title().set_position((10, 5)) 

# Add the first legend manually to the axes
ax1.add_artist(leg1)

# Create second legend for the line (without title)
ax1.legend(handles=[line], 
           loc='right',  # or adjust position as needed
           labelspacing=0.5, 
           frameon=False)

PlotGWorld.make_axes_gray(ax1)

# ===== PLOT 2: Bar plot of non-zero ratios =====
ax2 = axes[1]
data2 = ns_assertive_df[ns_assertive_df['comparison']=='gFeAR(Tier)-iFeAR']

# Calculate ratio
def calculate_nonzero_ratio(group):
    total_count = len(group)
    nonzero_count = (group['diff_n_assertive'] != 0).sum()
    return nonzero_count / total_count if total_count > 0 else 0

ratio_data = data2.groupby(['median_manhattan_dist', 'file']).apply(calculate_nonzero_ratio, include_groups=False).reset_index()
ratio_data.columns = ['median_manhattan_dist', 'file', 'nonzero_ratio']

sns.barplot(data=ratio_data, 
            x='median_manhattan_dist', 
            y='nonzero_ratio', 
            hue='file',
            hue_order=file_palette.keys(),
            saturation=1,
            palette=file_palette,
            ax=ax2)

# Define edge colors for each file
edge_color_dict = {'Aggressive':'black',
                   'Nuanced': 'white',
                   'Random': 'darkgoldenrod'}

# Apply edge colors to bars
containers = ax2.containers
file_list = list(file_palette.keys())

for file_idx, container in enumerate(containers):
    file_name = file_list[file_idx]
    edge_color = edge_color_dict[file_name]
    
    for bar in container:
        bar.set_edgecolor(edge_color)
        bar.set_linewidth(2)

ax2.set_xlabel('Median Manhattan Distance \nto affected agent', fontsize=28)
ax2.set_ylabel('Fraction of \n'+ r'non-zero $\Delta_j^{\mathrm{Assertive}}$',fontsize=28 )
# ax2.set_title(r'Ratio of non-zero $\Delta_j^{\mathrm{Assertive}}$ by median Manhattan distance')
ax2.set_title('')
# ax2.legend(loc='upper right')
ax2.legend().remove()


PlotGWorld.make_axes_gray(ax2)

# ===== PLOT 3: Mean difference =====
ax3 = axes[2]

# Seaborn can calculate error bars automatically using estimator and errorbar parameters
sns.barplot(data=data1,  # Use original data, not grouped
            x='median_manhattan_dist', 
            y='diff_n_assertive', 
            hue='file',
            hue_order=file_palette.keys(),
            saturation=1,
            palette=file_palette,
            estimator='mean',
            errorbar='se',  # 'sd' for standard deviation, 'se' for standard error, or ('ci', 95) for confidence interval
            capsize=0.1,
            ax=ax3)

# Apply edge colors to bars
containers = ax3.containers
file_list = list(file_palette.keys())

for file_idx, container in enumerate(containers):
    file_name = file_list[file_idx]
    edge_color = edge_color_dict[file_name]
    
    for bar in container:
        bar.set_edgecolor(edge_color)
        bar.set_linewidth(2)

ax3.set_xlabel('Median Manhattan distance\nto affected agent', fontsize=28)
ax3.set_ylabel(r'Mean $\Delta_j^{\mathrm{Assertive}}$'+' (+ SE)', fontsize=28)
# ax3.set_title(r'Mean $\Delta_j^{\mathrm{Assertive}}$ by median Manhattan distance')
ax3.set_title('')

legend = ax3.legend(loc='upper right', frameon=False)
# Then update legend to show edge colors
for legend_handle, file_name in zip(legend.legend_handles, file_palette.keys()):
    legend_handle.set_edgecolor(edge_color_dict[file_name])
    legend_handle.set_linewidth(1)


PlotGWorld.make_axes_gray(ax3)

ax1.spines[['right', 'top']].set_visible(False)
ax2.spines[['right', 'top']].set_visible(False)
ax3.spines[['right', 'top']].set_visible(False)

# Adjust layout and save
plt.tight_layout()

xticks_labels= ax3.get_xticklabels()
xticks_labels_int = [int(float(t.get_text())) for t in xticks_labels]
xticks = ax3.get_xticks()
ax3.set_xticks(xticks, labels=xticks_labels_int)



# Renaming Nuanced -> Directed
# Replace 'Nuanced' with 'Directed' in all legends across all axes
from matplotlib.legend import Legend
for ax in axes.flat:
    legends = [ax.legend_] + [child for child in ax.get_children() if isinstance(child, Legend)]
    for legend in legends:
        if legend is not None:
            for text in legend.get_texts():
                if text.get_text() == 'Nuanced':
                    text.set_text('Directed')


fig_name = 'Comparing-n-assertive-tier-fear-combined.pdf'
fig_path = os.path.join(fig_folder, fig_name)
plt.savefig(fig_path)

fig_name = 'Comparing-n-assertive-tier-fear-combined.png'
fig_path = os.path.join(fig_folder, fig_name)
plt.savefig(fig_path)

plt.show()

In [ ]:
# Bar plots - standard deviations of taus

sns.set_context("paper", font_scale=1.8)
g = sns.FacetGrid(taus_df, row='comparison', aspect=1., height=4)
g.map_dataframe(
    sns.barplot,
    x='median_manhattan_dist', y='tau',
    hue='file',
    hue_order=file_palette.keys(),
    saturation=1,
    palette=file_palette,
    estimator='std',
    errorbar=None,
    gap=0.1,
    capsize=0.1)

# 'Aggressive':'deeppink',
# 'Nuanced': 'deepskyblue',
# 'Random': 'gold'

# Define edge colors for each file
edge_color_dict = {'Aggressive':'black',
                   'Nuanced': 'white',
                   'Random': 'darkgoldenrod'}

# Apply edge colors to bars
for ax in g.axes.flat:
    containers = ax.containers
    file_list = list(file_palette.keys())
    
    for file_idx, container in enumerate(containers):
        file_name = file_list[file_idx]
        edge_color = edge_color_dict[file_name]
        
        for bar in container:
            bar.set_edgecolor(edge_color)
            bar.set_linewidth(1)


ax=g.axes[-1,0]
xticks_labels= ax.get_xticklabels()
xticks_labels_int = [int(float(t.get_text())) for t in xticks_labels]
xticks = ax.get_xticks()
ax.set_xticks(xticks, labels=xticks_labels_int)


# Add legend first
g.add_legend()

# Then update legend to show edge colors
for legend_handle, file_name in zip(g._legend.legend_handles, file_palette.keys()):
    legend_handle.set_edgecolor(edge_color_dict[file_name])
    legend_handle.set_linewidth(1)

# Renaming Nuanced -> Directed
for text in g._legend.get_texts():
    if text.get_text() == 'Nuanced':
        text.set_text('Directed')

g._legend.remove()

# g.set_titles("{row_name}")
g.set_titles("")
g.set_axis_labels('Median Manhattan distance\nto affected agent', 'Standard deviation of'+r'$\tau$')

for i, ax_row in enumerate(g.axes):
    row_name = g.row_names[i]
    row_name_formatted = row_name.replace("-", r" \\ ")
    comparison_text = rf'\left[ \substack{{{row_name_formatted}}} \right]'
    tau_y_label = r'$\tau$'+'\n'+ rf'${comparison_text}$'
    ylabel = f'SD of {tau_y_label}'
    # print(ylabel)
    ax_row[0].set_ylabel(ylabel)

plt.tight_layout()
PlotGWorld.make_axes_gray(g.axes[0,0])



fig_name = 'Comparing-std-of-taus.pdf'
fig_path = os.path.join(fig_folder,fig_name)
plt.savefig(fig_path)

In [ ]:
# Line plots - standard deviations of taus
sns.set_context("paper", font_scale=1.8)
g = sns.FacetGrid(taus_df, row='comparison', aspect=1.4, height=4)

# Define markers for each file
marker_dict = {
    'Aggressive': 'o',
    'Nuanced': 's',
    'Random': '^'
}

# Calculate std for each group
def plot_std_line(data, **kwargs):
    for file_name in file_palette.keys():
        file_data = data[data['file'] == file_name]
        grouped = file_data.groupby('median_manhattan_dist')['tau'].std().reset_index()
        
        plt.plot(
            grouped['median_manhattan_dist'], 
            grouped['tau'],
            marker=marker_dict[file_name],
            color=file_palette[file_name],
            markeredgecolor=edge_color_dict[file_name],
            markeredgewidth=1.5,
            markersize=8,
            linewidth=2,
            label=file_name
        )

g.map_dataframe(plot_std_line)

# Define edge colors for each file
edge_color_dict = {
    'Aggressive':'black',
    'Nuanced': 'white',
    'Random': 'darkgoldenrod'
}

# Set x-axis ticks
ax = g.axes[-1, 0]
xticks_labels = ax.get_xticklabels()
xticks_labels_int = [int(float(t.get_text())) for t in xticks_labels]
xticks = ax.get_xticks()
ax.set_xticks(xticks, labels=xticks_labels_int)

# Add legend
handles, labels = g.axes[0, 0].get_legend_handles_labels()
# Rename Nuanced -> Directed
labels = ['Directed' if label == 'Nuanced' else label for label in labels]
g.axes[-1, 0].legend(handles, labels)

# Set titles and labels
g.set_titles("")
g.set_axis_labels('Median Manhattan distance\nto affected agent', 'Standard deviation of'+r'$\tau$')

for i, ax_row in enumerate(g.axes):
    row_name = g.row_names[i]
    row_name_formatted = row_name.replace("-", r" \\ ")
    comparison_text = rf'\left[ \substack{{{row_name_formatted}}} \right]'
    tau_y_label = r'$\tau$'+'\n'+ rf'${comparison_text}$'
    ylabel = f'SD of {tau_y_label}'
    # print(ylabel)
    ax_row[0].set_ylabel(ylabel)

plt.tight_layout()
PlotGWorld.make_axes_gray(g.axes[0, 0])

fig_name = 'Comparing-std-of-taus-lines.pdf'
fig_path = os.path.join(fig_folder, fig_name)
plt.savefig(fig_path)